In [5]:
from pulp import *
import pandas as pd

data = pd.read_csv("data/cluster.csv", header=0)


# only existing vertiport and total profit columns
clusterName = data['label']
dataTable = data.iloc[:, -3:].values.tolist()
print(dataTable)

n_Ve = dict([(i, n[0]) for i, n in enumerate(dataTable)])
total_commute = dict([(i, n[1]) for i, n in enumerate(dataTable)])
total_prof = dict([(i, n[2]) for i, n in enumerate(dataTable)])

prob = LpProblem('MaxProfit', LpMinimize)

# number of existing vertiport
n_eVTOL_Vars = LpVariable.dicts('n_eVTOL', clusterName, 0, cat='Integer')
# number of new vertiport
n_Vn_Vars = LpVariable.dicts('n_Vn', clusterName, 0, cat='Integer')

n_round = 10
c_eVTOL = 1000000.0 ## updated to 1 million
c_Ve = 200000.0
c_Vn = 500000.0
cap_eVTOL = 4.0
cap_port = 8.0
depreciation_1 = 1/365
depreciation_10 = 1/365/10
depreciation_20 = 1/365/20

# Objection function
prob += lpSum([- total_prof[i] + c_eVTOL * n_eVTOL_Vars[i]*depreciation_10 + c_Ve * n_Ve[i]*depreciation_1 + c_Vn *
               n_Vn_Vars[i] * depreciation_20 for i in clusterName]), 'Total Profit'

# Constraint
for i in clusterName:
    prob += cap_eVTOL * n_eVTOL_Vars[i] * n_round >= total_commute[i]
    prob += cap_port * (n_Ve[i] + n_Vn_Vars[i]) >= n_eVTOL_Vars[i]

prob.solve()

print()

for var in prob.variables():
    print(str(var) + ' : ' + str(var.varValue))
print()

print("Total profit = $%.2f" % value(-prob.objective))
# res_dict = {'new_vertiport': [var.varValue[:10] for var in prob.variables()],
#             'eVTOL#': [var.varValue[10:] for var in prob.variables()]}
# res = pd.DataFrame(res_dict)
res = [var.varValue for var in prob.variables()]
res_df = pd.DataFrame(columns=['new_vertiport', 'eVTOL#'])
res_df['new_vertiport'] = res[:10]
res_df['eVTOL#'] = res[10:]
#res_df.to_csv('data/optimization_result.csv', index=False)

[[22, 2398, 23418], [4, 1948, 26239], [18, 2994, 36338], [3, 1210, 26221], [14, 2301, 25531], [12, 2317, 23761], [19, 1516, 23336], [5, 1469, 29907], [13, 2925, 33054], [49, 2224, 20626]]

n_Vn_0 : 0
n_Vn_1 : 3
n_Vn_2 : 0
n_Vn_3 : 1
n_Vn_4 : 0
n_Vn_5 : 0
n_Vn_6 : 0
n_Vn_7 : 0
n_Vn_8 : 0
n_Vn_9 : 0
n_eVTOL_0 : 60
n_eVTOL_1 : 49
n_eVTOL_2 : 75
n_eVTOL_3 : 31
n_eVTOL_4 : 58
n_eVTOL_5 : 58
n_eVTOL_6 : 38
n_eVTOL_7 : 37
n_eVTOL_8 : 74
n_eVTOL_9 : 56

Total profit = $34184.42


In [8]:
res_df['eVTOL#'].sum()

536

In [9]:
res_df

,new_vertiport,eVTOL#
0,0,60
1,3,49
2,0,75
3,1,31
4,0,58
5,0,58
6,0,38
7,0,37
8,0,74
9,0,56


In [11]:
value(-prob.objective) * 365

12477314.999999994

In [12]:
prob

MaxProfit:
MINIMIZE
68.4931506849315*n_Vn_0 + 68.4931506849315*n_Vn_1 + 68.4931506849315*n_Vn_2 + 68.4931506849315*n_Vn_3 + 68.4931506849315*n_Vn_4 + 68.4931506849315*n_Vn_5 + 68.4931506849315*n_Vn_6 + 68.4931506849315*n_Vn_7 + 68.4931506849315*n_Vn_8 + 68.4931506849315*n_Vn_9 + 273.972602739726*n_eVTOL_0 + 273.972602739726*n_eVTOL_1 + 273.972602739726*n_eVTOL_2 + 273.972602739726*n_eVTOL_3 + 273.972602739726*n_eVTOL_4 + 273.972602739726*n_eVTOL_5 + 273.972602739726*n_eVTOL_6 + 273.972602739726*n_eVTOL_7 + 273.972602739726*n_eVTOL_8 + 273.972602739726*n_eVTOL_9 + -181307.71232876708
SUBJECT TO
_C1: 40 n_eVTOL_0 >= 2398

_C2: 8 n_Vn_0 - n_eVTOL_0 >= -176

_C3: 40 n_eVTOL_1 >= 1948

_C4: 8 n_Vn_1 - n_eVTOL_1 >= -32

_C5: 40 n_eVTOL_2 >= 2994

_C6: 8 n_Vn_2 - n_eVTOL_2 >= -144

_C7: 40 n_eVTOL_3 >= 1210

_C8: 8 n_Vn_3 - n_eVTOL_3 >= -24

_C9: 40 n_eVTOL_4 >= 2301

_C10: 8 n_Vn_4 - n_eVTOL_4 >= -112

_C11: 40 n_eVTOL_5 >= 2317

_C12: 8 n_Vn_5 - n_eVTOL_5 >= -96

_C13: 40 n_eVTOL_6 >= 1516
